In [ ]:
%cd ../DenseCap_tf/

In [42]:
import tensorflow as tf
import numpy as np

import keras
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras.layers import Convolution2D, Lambda, Merge
import MakeAnchors, ReshapeBoxFeatures, ApplyBoxTransform

In [3]:
# load vgg16 model
base_model = VGG16(weights='imagenet')
model = Model(input=base_model.input, output=base_model.get_layer('block5_conv3').output)

In [9]:
# load random image
img_path = '/home/alex/FasterRCNN/py-faster-rcnn/data/demo/000456.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [10]:
# Create RPN with  Keras api
rpn = model(x)

In [11]:
# RPN Conv layer parameters
rpn_num_filters = 256
rpn_filter_size = 3
std = 0.01

# Add an extra conv layer and a ReLU
rpn = Convolution2D(rpn_num_filters, rpn_filter_size, rpn_filter_size, activation='relu',
                      border_mode='same', init='normal')(rpn)

In [15]:
# Set up anchor sizes
anchors = np.array([[45, 90], [90, 45], [64, 64],
                   [90, 180], [180, 90], [128, 128],
                   [181, 362], [362, 181], [256, 256],
                    [362, 724], [724, 362], [512, 512]]).T

num_anchors = anchors.shape[1]

In [16]:
# Branch to produce box coordinates for each anchor
# This branch will return {boxes, {anchors, transforms}}
# Box branch Conv layer parameters
box_input_shape = 14, 14, 256
box_num_filters = 4 * num_anchors
box_filter_size = 1

box_branch = Convolution2D(box_num_filters, box_filter_size, box_filter_size, 
                             border_mode='valid', init='normal')(rpn)

In [25]:
# Branch to make anchors
make_anchors_branch = Lambda(MakeAnchors.make_anchors)(box_branch)
make_anchors_branch = ReshapeBoxFeatures.ReshapeBoxFeatures(num_anchors)(make_anchors_branch)

In [28]:
# Intermediate layer, just to reshape a tensor for ReshapeBoxFeatures layer
def tensor_reshape(tensor):
    tensor_shape = tensor.get_shape().as_list()
    return K.reshape(tensor,
                     [tensor_shape[0], tensor_shape[3], tensor_shape[1], tensor_shape[2]])

In [32]:
# Branch that creates offsets for boxes, cannot understand how it works
reshape_boxes_branch = Lambda(tensor_reshape)(box_branch)
reshape_boxes_branch = ReshapeBoxFeatures.ReshapeBoxFeatures(num_anchors)(reshape_boxes_branch)

In [33]:
# Merge make_anchors_branch and reshape_boxes_branch together into box_branch
box_branch = Merge(
    mode=lambda x: tf.stack(x)
                         )([make_anchors_branch, reshape_boxes_branch])

In [40]:
K.get_value(box_branch)[0].shape

(2352, 4)

In [43]:
# Apply adjustments to bounding boxes for bounding box regression
box_branch = ApplyBoxTransform.ApplyBoxTransform()(box_branch)

ValueError: slice index 2 of dimension 0 out of bounds. for 'strided_slice_4' (op: 'StridedSlice') with input shapes: [2,2352,4], [1], [1], [1].